# AI Service End-to-End Testing

This notebook tests the multi-stage vision analysis pipeline.

**Pipeline stages:**
1. Content safety check (NSFW detection)
2. Species detection (dog/cat)
3. Breed classification (with crossbreed detection)
4. RAG enrichment (breed knowledge)
5. Contextual Ollama visual analysis

**Prerequisites:**
- All services running: `docker compose ps`
- AI service healthy: `docker exec ft_transcendence_ai_service curl http://localhost:3003/health`

In [ ]:
import requests
import json
import base64
from io import BytesIO
from PIL import Image
from IPython.display import display, HTML
import pandas as pd

# AI Service endpoint (internal, via Docker network)
AI_SERVICE_URL = "http://ai-service:3003"

print("✅ Imports loaded successfully")

## 1. Create Test Image

Generate a simple test image for the pipeline.

In [ ]:
def create_test_image(color="brown", size=(512, 512)):
    """Create a test image and return as base64 data URI."""
    img = Image.new('RGB', size, color=color)
    buffer = BytesIO()
    img.save(buffer, format='JPEG')
    encoded = base64.b64encode(buffer.getvalue()).decode()
    return f"data:image/jpeg;base64,{encoded}"

# Create test image
test_image = create_test_image("brown")
print(f"✅ Test image created: {len(test_image)} bytes")
print(f"   Preview: {test_image[:100]}...")

# Display the image
img_data = base64.b64decode(test_image.split(',')[1])
display(Image.open(BytesIO(img_data)))

## 2. Test Full Vision Analysis Pipeline

Send the image through the complete multi-stage pipeline.

In [ ]:
# Make request to vision analysis endpoint
response = requests.post(
    f"{AI_SERVICE_URL}/api/v1/vision/analyze",
    json={"image": test_image},
    timeout=60
)

print(f"Status Code: {response.status_code}")
print(f"\nRaw Response:")
result = response.json()
print(json.dumps(result, indent=2))

## 3. Parse and Display Results

Extract key information from the pipeline response.

In [ ]:
if result.get('success'):
    data = result['data']
    
    print("="*60)
    print("🐕 VISION ANALYSIS RESULTS")
    print("="*60)
    
    # Species
    print(f"\n📊 Species: {data['species']}")
    
    # Breed Analysis
    breed = data['breed_analysis']
    print(f"\n🐾 Breed Analysis:")
    print(f"   Primary Breed: {breed['primary_breed']}")
    print(f"   Confidence: {breed['confidence']:.2%}")
    print(f"   Crossbreed: {breed['is_likely_crossbreed']}")
    
    # Breed probabilities
    if breed['breed_probabilities']:
        print(f"\n   Top Breeds:")
        for bp in breed['breed_probabilities'][:3]:
            print(f"      - {bp['breed']}: {bp['probability']:.2%}")
    
    # Crossbreed info
    if breed['crossbreed_analysis']:
        cross = breed['crossbreed_analysis']
        print(f"\n   Crossbreed Details:")
        print(f"      Common Name: {cross.get('common_name', 'Unknown')}")
        print(f"      Parent Breeds: {', '.join(cross['detected_breeds'])}")
        print(f"      Reasoning: {cross['confidence_reasoning']}")
    
    # Visual Description
    print(f"\n👁️  Visual Description:")
    print(f"   {data['description']}")
    
    # Traits
    traits = data['traits']
    print(f"\n🎯 Traits:")
    print(f"   Size: {traits['size']}")
    print(f"   Energy Level: {traits['energy_level']}")
    print(f"   Temperament: {traits['temperament']}")
    
    # Health Observations
    print(f"\n💊 Health Observations:")
    if data['health_observations']:
        for obs in data['health_observations']:
            print(f"   • {obs}")
    else:
        print("   None noted")
    
    # RAG Enrichment
    if data.get('enriched_info'):
        enriched = data['enriched_info']
        print(f"\n📚 RAG Enrichment:")
        print(f"   Breed: {enriched.get('breed', 'N/A')}")
        if enriched.get('parent_breeds'):
            print(f"   Parent Breeds: {', '.join(enriched['parent_breeds'])}")
        print(f"   Description: {enriched.get('description', 'N/A')[:200]}...")
        print(f"   Care: {enriched.get('care_summary', 'N/A')[:150]}...")
        print(f"   Health Info: {enriched.get('health_info', 'N/A')[:150]}...")
        print(f"   Sources: {', '.join(enriched.get('sources', []))}")
    else:
        print(f"\n⚠️  RAG enrichment unavailable (graceful degradation)")
    
    print("\n" + "="*60)
    
else:
    print("❌ Analysis Failed")
    print(f"Error: {result.get('error', 'Unknown error')}")

## 4. Test Error Handling

Verify pipeline rejection scenarios.

In [ ]:
# Test with invalid image
print("Testing invalid image handling...")
invalid_response = requests.post(
    f"{AI_SERVICE_URL}/api/v1/vision/analyze",
    json={"image": "data:image/jpeg;base64,invalid_data"},
    timeout=30
)

print(f"Status: {invalid_response.status_code}")
print(f"Response: {json.dumps(invalid_response.json(), indent=2)}")

## 5. Performance Benchmark

Measure pipeline latency.

In [ ]:
import time

print("Running performance benchmark (3 requests)...\n")

latencies = []
for i in range(3):
    start = time.time()
    response = requests.post(
        f"{AI_SERVICE_URL}/api/v1/vision/analyze",
        json={"image": test_image},
        timeout=60
    )
    latency = time.time() - start
    latencies.append(latency)
    print(f"Request {i+1}: {latency:.2f}s (status: {response.status_code})")

print(f"\nAverage Latency: {sum(latencies)/len(latencies):.2f}s")
print(f"Min: {min(latencies):.2f}s, Max: {max(latencies):.2f}s")

## 6. Service Health Checks

Verify all pipeline components are operational.

In [ ]:
print("Checking service health...\n")

services = [
    ("AI Service", "http://ai-service:3003/health"),
    ("Vision Route", "http://ai-service:3003/api/v1/vision/health"),
    ("Classification Service", "http://classification-service:3004/health"),
    ("Ollama", "http://ollama:11434/api/tags"),
]

health_results = []
for name, url in services:
    try:
        resp = requests.get(url, timeout=5)
        status = "✅ Healthy" if resp.status_code == 200 else "⚠️  Degraded"
        health_results.append({"Service": name, "Status": status, "Code": resp.status_code})
    except Exception as e:
        health_results.append({"Service": name, "Status": "❌ Down", "Code": str(e)})

df = pd.DataFrame(health_results)
display(df)

## Summary

This notebook demonstrates:
- ✅ Multi-stage pipeline execution
- ✅ Species and breed classification
- ✅ Crossbreed detection
- ✅ RAG enrichment with breed knowledge
- ✅ Contextual visual analysis
- ✅ Error handling and validation
- ✅ Performance benchmarking

**Next Steps:**
- Test with real pet images
- Validate crossbreed detection accuracy
- Monitor pipeline latency in production